<a href="https://colab.research.google.com/github/gtmray/face_mask/blob/master/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/chandrikadeb7/Face-Mask-Detection.git

Cloning into 'Face-Mask-Detection'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 3914 (delta 0), reused 0 (delta 0), pack-reused 3913
Receiving objects: 100% (3914/3914), 148.67 MiB | 8.95 MiB/s, done.
Resolving deltas: 100% (73/73), done.
Checking out files: 100% (3870/3870), done.


In [2]:
import shutil
shutil.move('/content/Face-Mask-Detection/dataset', '/content')
shutil.rmtree('/content/Face-Mask-Detection/')

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os, random

In [4]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [5]:
os.chdir('/content')
os.makedirs('test/with_mask')
os.makedirs('test/without_mask')

def change_dir_test(dirpath, destDirectory):
  filenames = random.sample(os.listdir(dirpath), 150)
  for fname in filenames:
      srcpath = os.path.join(dirpath, fname)
      shutil.move(srcpath, destDirectory)
  os.chdir('/content')

dirpath_wm_test = '/content/dataset/with_mask'
destDirectory_wm_test = 'test/with_mask'
change_dir_test(dirpath_wm_test, destDirectory_wm_test)

dirpath_wt_test = '/content/dataset/without_mask'
destDirectory_wt_test = 'test/without_mask'
change_dir_test(dirpath_wt_test, destDirectory_wt_test)

In [7]:
os.chdir('/content')
os.makedirs('train/with_mask')
os.makedirs('train/without_mask')

dirpath_wm_train = '/content/dataset/with_mask'
destDirectory_wm_train = 'train/with_mask'

dirpath_wt_train = '/content/dataset/without_mask'
destDirectory_wt_train = 'train/without_mask'

def change_dir_train(dirpath, destDirectory):
  filenames = os.listdir(dirpath)
  for fname in filenames:
      srcpath = os.path.join(dirpath, fname)
      shutil.move(srcpath, destDirectory)
  os.chdir('/content')

change_dir_train(dirpath_wm_train, destDirectory_wm_train)
change_dir_train(dirpath_wt_train, destDirectory_wt_train)

In [10]:
#Data preprocessing

train_path = '/content/train'
valid_path = '/content/test'

train_batches = ImageDataGenerator(preprocessing_function=keras.applications.vgg16.preprocess_input).flow_from_directory(directory=train_path, target_size=(120, 120), classes=['with_mask', 'without_mask'], batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=keras.applications.vgg16.preprocess_input).flow_from_directory(directory=valid_path, target_size=(120, 120), classes=['with_mask', 'without_mask'], batch_size=10)

"""CREATING A CNN MODEL"""

model = Sequential([
                    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(100, 100, 3)),
                    MaxPool2D(pool_size=(2, 2), strides=2),
                    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
                    MaxPool2D(pool_size=(2, 2), strides=2),
                    Flatten(),
                    Dense(units=2, activation='softmax')
])

model.summary()

"""TRAINING THE MODEL"""

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(x=train_batches ,validation_data=valid_batches, epochs=25, verbose=2)
model.save('model.h5')

Found 3546 images belonging to 2 classes.
Found 300 images belonging to 2 classes.


'CREATING A CNN MODEL'